In [1]:
#!pip install plotly

import json
import numpy as np
import pandas as pd
import plotly.express as px
from unidecode import unidecode
import plotly.io as pio
pio.renderers.default = 'browser'

In [2]:
json.__version__

'2.0.9'

In [3]:
municipios_rs = json.load(open("municipios_rs.geojson", "r"))

In [4]:
municipios_rs

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'id': '4300034', 'name': 'Aceguá', 'description': 'Aceguá'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-54.112761022, -31.429314841],
      [-54.0982072825, -31.4365537224],
      [-54.0933016419, -31.4358827699],
      [-54.0933041073, -31.4468651549],
      [-54.0704382482, -31.4560808987],
      [-54.0680920618, -31.4721024925],
      [-54.0480591947, -31.4877431516],
      [-54.040362385, -31.4996841144],
      [-54.0295423551, -31.5018570121],
      [-54.0277520782, -31.5062144318],
      [-54.0343962893, -31.5141476175],
      [-54.0326835947, -31.5175941725],
      [-54.040689142, -31.5225946346],
      [-54.0138603201, -31.537109864],
      [-54.0086121661, -31.526602478],
      [-53.994405446, -31.5255438331],
      [-53.9865355619, -31.5150191469],
      [-53.9801718373, -31.5137295236],
      [-53.9666009538, -31.4983828925],
      [-53.9570879085, -31.4933159515],
      [-53.968

In [5]:
#municipios_rs['features'][0]

In [6]:
#municipios_id_map = {}
#for feature in municipios_rs['features']:
#    feature['id'] = feature['properties']['id']
#    municipios_id_map[feature['properties']['name']] = feature['id']

In [5]:
municipios_id_map = {}
for feature in municipios_rs['features']:
    feature['id'] = feature['properties']['name']
    municipios_id_map[feature['properties']['description']] = feature['id']

In [6]:
municipios_rs['features'][0]['properties']

{'id': '4300034', 'name': 'Aceguá', 'description': 'Aceguá'}

In [7]:
# Função remover acentos
def strip_accents(text):
    return str(unidecode(text))

# Arquivo com as lat e long
df = pd.read_excel('eapv_mun_geo.xlsx')

# apaga dados do BE passado:
df_mun_geo = df[['MUNICIPIO', 'CRS', 'latitude', 'longitude']]

# Incidência de EAPV por Município
df_mun_incid = pd.read_excel('municipios_incidencia_EAPV.xlsx')

# Inserir latitude e longitude por municipio
# cria coluna MUNICIPIO para merge
df_mun_incid['MUNICIPIO'] = df_mun_incid['municipio'].apply(lambda x: strip_accents(x).upper())
df_mun_incid

# Merge 
df_mun_geo_incid = pd.merge(df_mun_incid, df_mun_geo, on='MUNICIPIO', how='left')
df_mun_geo_incid = df_mun_geo_incid[['MUNICIPIO', 'municipio', 'incidencia_1000ds', 'latitude', 'longitude']]
df_mun_geo_incid.loc[df_mun_geo_incid['municipio'].str.contains('Santana do Livramento'), 'municipio'] = "Sant' Ana do Livramento"

print(df_mun_geo_incid.shape)
# Tirando a cidade de Pinto Bandeira do mapa (pois da erro e não tem no painel do rs)
df_mun_geo_incid.drop([317], inplace=True)
print(df_mun_geo_incid.shape)

# Trocando o nome da coluna incidencia
df_mun_geo_incid.rename(columns={'incidencia_1000ds': 'Incidência'}, inplace = True)


df_mun_geo_incid['id'] = df_mun_geo_incid['municipio'].apply(lambda x: municipios_id_map[x])
df_mun_geo_incid.head(5)

(497, 5)
(496, 5)


,MUNICIPIO,municipio,Incidência,latitude,longitude,id
0,ACEGUA,Aceguá,0.45,-31.8665,-54.1615,Aceguá
1,AGUA SANTA,Água Santa,0.00,-28.1672,-52.0310,Água Santa
2,AGUDO,Agudo,0.10,-29.6447,-53.2515,Agudo
3,AJURICABA,Ajuricaba,0.40,-28.2342,-53.7757,Ajuricaba
4,ALECRIM,Alecrim,0.32,-27.6579,-54.7649,Alecrim


In [12]:
df_mun_geo_incid.sort_values(['Incidência']).tail()

,MUNICIPIO,municipio,Incidência,latitude,longitude,id
321,POCO DAS ANTAS,Poço das Antas,6.15,-29.4481,-51.6719,Poço das Antas
291,OSORIO,Osório,6.17,-29.8881,-50.2667,Osório
120,COXILHA,Coxilha,8.86,-28.1280,-52.3023,Coxilha
289,NOVO XINGU,Novo Xingu,11.55,-27.7490,-53.0639,Novo Xingu
133,DOIS LAJEADOS,Dois Lajeados,13.20,-28.9830,-51.8396,Dois Lajeados


In [8]:
# teste 1
#fig = px.choropleth(
#    df_mun_geo_incid,
#    locations="id",
#    geojson=municipios_rs,
#    color="Incidência",
#    hover_name="municipio",
#    hover_data=["Incidencia"],
#    title="teste mapa",
#)
#fig.update_geos(fitbounds="locations", visible=False)
#fig.show()

In [16]:
# teste 2
fig = px.choropleth_mapbox(
    df_mun_geo_incid,
    locations="id",
    geojson=municipios_rs,
    color="Incidência",
    #hover_name="municipio",
    hover_data=["Incidência"],
    title="Incidência de EAPV pós-vacinação contra COVID-19, por 1.000 doses aplicadas, RS, 18/01/2021-05/02/2022 - Boletim 01/2022",
    #mapbox_style="carto-positron",
    mapbox_style='open-street-map',
    center={"lat": -30, "lon": -53},
    zoom=6,
    opacity=1,
    color_continuous_scale=px.colors.sequential.Oranges,
    color_continuous_midpoint=10,
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [17]:
# Salvar a figura
fig.write_html('Mapa - Inciência de EAPV no Rio Grande do Sul BE 01_2022.html', auto_open=False)